In [68]:
import os
import pandas as pd
import seaborn as sns
from PIL import Image
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import visualkeras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pydot
import graphviz
import tensorflow as tf

In [ ]:
root_dir = 'C:/Users/nguye/Downloads/Traffic and Road Signs.v1i.multiclass'

train_dir = root_dir + '/train/'
val_dir = root_dir + '/valid/'
# test_dir = root_dir + '/test/'
test_dir = root_dir + '/train/'

train_df = pd.read_csv(train_dir + '/_classes.csv')
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)
val_df = pd.read_csv(val_dir + '/_classes.csv')
# test_df = pd.read_csv(test_dir + '/_classes.csv')

In [ ]:
print(len(train_df))
print(len(val_df))

7092
1884


In [135]:
class_columns =  [" -Road narrows on right",
                " 50 mph speed limit",
                " Attention Please-",
                " Beware of children",
                " CYCLE ROUTE AHEAD WARNING",
                " Dangerous Left Curve Ahead",
                " Dangerous Rright Curve Ahead",
                " End of all speed and passing limits",
                " Give Way",
                " Go Straight or Turn Right",
                " Go straight or turn left",
                " Keep-Left",
                " Keep-Right",
                " Left Zig Zag Traffic",
                " No Entry",
                " No_Over_Taking",
                " Overtaking by trucks is prohibited",
                " Pedestrian Crossing",
                " Round-About",
                " Slippery Road Ahead",
                " Speed Limit 20 KMPh",
                " Speed Limit 30 KMPh",
                " Stop_Sign",
                " Straight Ahead Only",
                " Traffic_signal",
                " Truck traffic is prohibited",
                " Turn left ahead",
                " Turn right ahead",
                " Uneven Road"]

In [136]:
# Turn filenames into paths

train_df['filename'] = train_dir + train_df['filename']
train_df = train_df.rename(columns={'filename': 'path'})

val_df['filename'] = val_dir + val_df['filename']
val_df = val_df.rename(columns={'filename': 'path'})

test_df['filename'] = test_dir + test_df['filename']
test_df = test_df.rename(columns={'filename': 'path'})
pass

In [137]:
train_paths = train_df['path'].values
train_labels = train_df[class_columns].values

val_paths = val_df['path'].values
val_labels = val_df[class_columns].values

test_paths = test_df['path'].values
test_labels = test_df[class_columns].values

def load_image(filepath, label):
    image = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128))
    image = image / 255.0
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_image).batch(32)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(load_image).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_image).batch(32)

In [138]:
train_ds = train_ds.shuffle(buffer_size=32)

model = Sequential([
#   layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(class_columns))
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [139]:
epochs=5
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/5


178/178 [==============================] - 14s 74ms/step - loss: 1.5058 - accuracy: 0.5865 - val_loss: 1.3153 - val_accuracy: 0.8418
Epoch 2/5
178/178 [==============================] - 13s 72ms/step - loss: 0.2738 - accuracy: 0.9372 - val_loss: 1.1428 - val_accuracy: 0.9050
Epoch 3/5
178/178 [==============================] - 13s 72ms/step - loss: 0.0995 - accuracy: 0.9774 - val_loss: 1.0911 - val_accuracy: 0.9220
Epoch 4/5
178/178 [==============================] - 12s 69ms/step - loss: 0.0463 - accuracy: 0.9908 - val_loss: 0.9390 - val_accuracy: 0.9183
Epoch 5/5
178/178 [==============================] - 13s 72ms/step - loss: 0.0332 - accuracy: 0.9945 - val_loss: 1.2852 - val_accuracy: 0.9199


In [140]:
model.evaluate(test_ds, batch_size=32)

NotFoundError: Graph execution error:

NewRandomAccessFile failed to Create/Open: C:/Users/nguye/Downloads/Traffic and Road Signs.v1i.multiclass/test/00040_00009_00011_png_jpg.rf.b719cb0274f20b63492974a2fda8d85b.jpg : The system cannot find the file specified.
; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_test_function_58242]